In [1]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt


In [2]:

# Initialize wandb API
api = wandb.Api()

# Replace 'your_sweep_id' with your actual sweep ID
sweep_id = 'maximes_crew/S3_SHD_runs/pqc7ir1o'

# Fetch the sweep object
sweep = api.sweep(sweep_id)

# Fetch all runs in the sweep
runs = sweep.runs

sweep_params = ['lr', 'pdrop', 'scheduler_patience', 'scheduler_factor', 'dt_max']

In [3]:
runs

<Runs maximes_crew/S3_SHD_runs>

In [4]:
# Initialize an empty DataFrame to hold the relevant data
data = []

# Extract the necessary data from each run
for run in runs:
    config = run.config
    summary = run.summary
    n_layers = config.get('nb_layers')
    n_hiddens = config.get('nb_hiddens')
    lr = config.get('lr')
    dt_min = config.get('dt_min')
    dt_max = config.get('dt_max')
    pdrop = config.get('pdrop')
    sched_patience = config.get('scheduler_patience')
    sched_factor = config.get('scheduler_factor')
    best_valid_acc = summary.get('best valid acc')

    if best_valid_acc is not None:
        data.append({
            'n_layers': n_layers,
            'n_hiddens': n_hiddens,
            'lr': lr,
            'dt_min': dt_min,
            'dt_max': dt_max,
            'pdrop' : pdrop,
            'scheduler_patience' : sched_patience,
            'scheduler_factor' : sched_factor,
            'best_valid_acc': best_valid_acc
        })



In [5]:
data

[{'n_layers': 3,
  'n_hiddens': 128,
  'lr': 0.005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.1,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'best_valid_acc': 0.9384864267676768},
 {'n_layers': 3,
  'n_hiddens': 128,
  'lr': 0.005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.1,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'best_valid_acc': 0.9456281565656564},
 {'n_layers': 3,
  'n_hiddens': 128,
  'lr': 0.005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.1,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'best_valid_acc': 0.9393544823232324},
 {'n_layers': 3,
  'n_hiddens': 128,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'scheduler_patience': 10,
  'scheduler_factor': 0.9,
  'best_valid_acc': 0.9415246212121212},
 {'n_layers': 3,
  'n_hiddens': 128,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'scheduler_patience': 10,
  'scheduler_factor': 0.9,
  'best_valid_acc': 0.942589962121212},
 {'n_layers

In [6]:
# Convert the data to a DataFrame for easier manipulation
df = pd.DataFrame(data)

# Group by n_layers and lr, then compute the mean of best_valid_acc for each group
df_grouped = df.groupby(sweep_params).agg({'best_valid_acc': 'mean'}).reset_index()

# Calculate the number of runs for each line configuration
df['run_count'] = df.groupby(sweep_params).transform('size')

# Add the run count to the grouped dataframe
df_grouped['run_count'] = df_grouped[sweep_params].apply(lambda x: df[(df[sweep_params] == x).all(axis=1)]['run_count'].iloc[0], axis=1)

df_grouped_sorted = df_grouped.sort_values(by='best_valid_acc', ascending=False).reset_index(drop=True)

In [7]:
df_grouped_sorted[0:10]

,lr,pdrop,scheduler_patience,scheduler_factor,dt_max,best_valid_acc,run_count
0,0.01,0.50,5,0.9,0.5,0.950560,5
1,0.01,0.10,10,0.9,0.5,0.949345,5
2,0.01,0.25,10,0.7,0.5,0.949100,5
3,0.01,0.50,10,0.9,0.5,0.949029,5
4,0.01,0.10,5,0.9,0.5,0.948595,5
5,0.01,0.25,10,0.9,0.5,0.947246,5
6,0.01,0.50,10,0.7,0.5,0.947238,5
7,0.01,0.10,5,0.7,0.5,0.944010,5
8,0.01,0.10,10,0.7,0.5,0.943955,5
9,0.01,0.25,5,0.9,0.5,0.943158,5
